In [1]:
import numpy as np

In [2]:
def compute_M_at_idx(
    M: np.ndarray,
    Ix: np.ndarray,
    Iy: np.ndarray,
    i: int,
    j: int,
    v: str,
    w: str,
    scoring_matrix: dict,
):
    if i == 0 and j == 0:
        M[i][j] = 0
    elif i == 0 or j == 0:
        M[i][j] = -np.inf
    else:
        try:
            score = scoring_matrix[v[i - 1]][w[j - 1]]
        except KeyError:
            score = scoring_matrix[w[j - 1]][v[i - 1]]

        M[i][j] = max(
            M[i - 1][j - 1] + score, Ix[i - 1][j - 1] + score, Iy[i - 1][j - 1] + score
        )


def compute_Ix_at_idx(
    M: np.ndarray,
    Ix: np.ndarray,
    i: int,
    j: int,
    gap_open_penalty: int,
    gap_extend_penalty: int,
):
    if i == 0 and j == 0:
        Ix[i][j] = gap_open_penalty
    elif i == 0:
        Ix[i][j] = -np.inf
    else:
        Ix[i][j] = max(
            M[i - 1][j] + gap_open_penalty + gap_extend_penalty,
            Ix[i - 1][j] + gap_extend_penalty,
        )


def compute_Iy_at_idx(
    M: np.ndarray,
    Iy: np.ndarray,
    i: int,
    j: int,
    gap_open_penalty: int,
    gap_extend_penalty: int,
):
    if i == 0 and j == 0:
        Iy[i][j] = gap_open_penalty
    elif j == 0:
        Iy[i][j] = -np.inf
    else:
        Iy[i][j] = max(
            M[i][j - 1] + gap_open_penalty + gap_extend_penalty,
            Iy[i][j - 1] + gap_extend_penalty,
        )


def compute_matrices(
    v: str,
    w: str,
    scoring_matrix: dict,
    gap_open_penalty: int,
    gap_extend_penalty: int,
):
    n = len(v)
    m = len(w)

    M = np.zeros((n + 1, m + 1))
    Ix = np.zeros((n + 1, m + 1))
    Iy = np.zeros((n + 1, m + 1))

    for i in range(n + 1):
        for j in range(m + 1):
            compute_M_at_idx(M, Ix, Iy, i, j, v, w, scoring_matrix)
            compute_Ix_at_idx(M, Ix, i, j, gap_open_penalty, gap_extend_penalty)
            compute_Iy_at_idx(M, Iy, i, j, gap_open_penalty, gap_extend_penalty)

    return M, Ix, Iy

In [3]:
v = "ACACT"
w = "AAT"

scoring_matrix = {
    "A": {"A": 2, "C": -1, "G": 0, "T": -1},
    "C": {"C": 2, "G": -1, "T": 0},
    "G": {"G": 2, "T": -1},
    "T": {"T": 2},
}

gap_open_penalty = -3
gap_extend_penalty = -1

M, Ix, Iy = compute_matrices(w, v, scoring_matrix, gap_open_penalty, gap_extend_penalty)

In [4]:
M

array([[  0., -inf, -inf, -inf, -inf, -inf],
       [-inf,   2.,  -5.,  -3.,  -7.,  -8.],
       [-inf,  -2.,   1.,   0.,  -4.,  -5.],
       [-inf,  -6.,  -2.,   0.,   0.,  -2.]])

In [5]:
Ix

array([[ -3., -inf, -inf, -inf, -inf, -inf],
       [ -4., -inf, -inf, -inf, -inf, -inf],
       [ -5.,  -2.,  -9.,  -7., -11., -12.],
       [ -6.,  -3.,  -3.,  -4.,  -8.,  -9.]])

In [6]:
Iy

array([[ -3.,  -4.,  -5.,  -6.,  -7.,  -8.],
       [-inf, -inf,  -2.,  -3.,  -4.,  -5.],
       [-inf, -inf,  -6.,  -3.,  -4.,  -5.],
       [-inf, -inf, -10.,  -6.,  -4.,  -4.]])